# Fest R-CNN 

CNN fineTuning, bounding box regression, classification을 모두 하나의 
네트워크에서 학습시키는 end-to-end 기법 제시

성능 : pascal VOC 2007을 기준으로 66%

## Fest R-CNN이 Object-Detection을 수행하는 알고리즘 

1. 먼저 전체 이미지를 미리 학습된 CNN을 통과시켜 피쳐맵을 추출합니다.
2. Selective Search를 통해서 찾은 각각의 RoI에 대하여 RoI(Region of Interest) Pooling을 진행함. 고정된 크기의 feature vector를 얻음
3. feature vector는 fully connected layer들을 통과한 뒤, 두개의 브랜치로 나뉨
4-1. 하나의 브랜치는 softmax를 통과하여 해당 RoI가 어떤물체인지 classification. 더 이상 svm은 사용되지 않음
4-2. 나머지는 bounding box regression을 통해서 Selective Search로 찾은 박스의 위치를 조정함


### 1. RoI Pooling

CNN을 통과하여 추출된 피처맵을 미리 정해 놓은 H x W에 맞게 그리드를 설정한다.
각각의 칸 별로 가장 큰 값을 추출하는 max Pooling을 실시하면 결과값은 항상 H x W 크기의 피쳐맵이되고 
이를 쫙펴서 feature vector를 추출한다. 

- 인풋이미지와 피쳐맵의 크기가 다를경우 그 비율을 구해 RoI를 조절한다음 RoI 값을 조절한 후 진행

### 2. Multi Task Loss

고정 된 크기의 feature Vector를 구했으니, 각각 Classification, bounding box regression을 적용하여 각각의 Loss를 얻어내고
이를 Back Propagation 하여 전체모델을 학습시키면 된다.

이 때, Classification Loss, bounding box regression을 적절하게 엮어주는 것이 필요한데, 이를 Multi Task Loss 라고 한다.

        L(p, u, t^u, v) = L_cls(p, u) + lambda\[u >= 1\]L_loc(t^u, v)


먼저 입력으로 

softmax

        p = softmax를 통해서 얻어낸 K + 1 (k object + background)개의 확률 값 | p = (p_0, ..., p_k)

        u = 해당 RoI의 Ground Truth 라벨 값

bounding box regression 

k+1개 클래스에 대하여 각각 x, y, w, h 값을 조정하는 tk를 리턴한다. loss function 에는 이들 값 가운데 Ground Truth 라벨에 해당되는 값만 갖고오며 이는 tu에 해당

        t^u_n = (t^u_nx, t^u_ny, t^u_nw, t^u_nh) // 결과중 일부

다시 전체로스로 돌아가면 앞부분은 p, u를 갖고 Classification loss를 구한다. 뒷부분은 bounding box regression 을 통해 얻는다.

        L_cls(p,u) = -logp_u

        L_loc(t^u, v) = sigma(i는 {x, y, w, h}의 부분집합) smooth_L1(t^u_i - v_i)

 smooth_L1(t^u_i - v_i) :
 예측값과 라벨값의 차이를 계산후 smooth_L1이라는 함수를 통과시킨 합을 계산한다.

        smooth_L1(x) = 0.5x^2 (if |x| <1), |x| - 0.5 (otherwise)  




해당 논문은 object detection 테스크를 푸는 end-to-end 모델을 제시하면서 학습 단계를 간소화시키고 정확도와 성능 모두를 향상시켰다는 의의가 있다.

그러나 여전히 region proposal을 selective search로 수행하고, 이는 CPU 연산으로만 수행 가능하다는 한계점이 있습니다. 이 부분을 제외하면 inference에 소요되는 시간이 0.3초 정도로 짧다.

참고 

[1] https://yeomko.tistory.com/15

[2] https://gist.github.com/aisolab/46f73657c7fcbec91004708e01a67ba0